In [5]:
import pickle
import pandas as pd
import networkx as nx

In [6]:
with open(
    "/home/mel/Desktop/repos/cottagecore_tradfem_pipeline/pickle_files/full_data2.pkl",
    "rb",
) as f:
    df = pickle.load(f)
print(df.head())
print(len(df))

               time_posted           blog_name             post_id post_type  \
0  2022-04-24 19:20:26 GMT          loftyangel  682442648358961152     photo   
1  2022-04-24 19:20:23 GMT          happyheidi  682442645664055296      text   
2  2022-04-24 19:05:25 GMT  herartificialworld  682441703660175360      text   
3  2022-04-24 18:18:47 GMT           ddreiberg  682438769880793088      text   
4  2022-04-24 18:11:37 GMT       0-solshroom-0  682438319111602176      text   

                                          image_urls  \
0                                                 []   
1  [https://64.media.tumblr.com/892e4da6414ce00b1...   
2  [https://64.media.tumblr.com/840696a8019cceb54...   
3  [https://64.media.tumblr.com/5cb21a9d5de58b8af...   
4                                                 []   

                                               links  \
0                                                 []   
1  [https://moodcafe.tumblr.com/post/660065999662...   
2             

In [15]:
# getting tradwife tags thru co-occurence network

# start with all the posts tagged tradwife
contains = ["tradwife" in tags for tags in df["tags"]]
tradwifedf = df[contains]
print(len(tradwifedf))


# now build a network with tags from those posts
def make_cooccurence(df):
    """
    takes a df of tumblr posts (cleaned by cleandata2.py) and returns a networkx tag co-occurence graph

    pandas df -> networkx graph
    """
    T = nx.Graph()

    for index, row in df.iterrows():  # loop thru posts
        for tag1 in row["tags"]:
            for tag2 in row["tags"]:  # for each combination of tags within a post
                if tag1 != tag2:  # if they are not the same tag
                    if T.has_edge(tag1, tag2):
                        T[tag1][tag2]["weight"] = T[tag1][tag2]["weight"] + 1
                    else:
                        T.add_edge(tag1, tag2, weight=1)
    print("Num nodes: " + str(len(T.nodes())))
    print("Num edges: " + str(len(T.edges())))
    return T


T = make_cooccurence(tradwifedf)

1557
Num nodes: 746
Num edges: 11545


In [33]:
T_trimmed = nx.k_core(T, k=33)  # trimming to 33rd k-core and choosing out of this list
print(len(T_trimmed.nodes()))
print(T_trimmed.nodes())
tradtags = [
    "traditional femininity",
    "traditional gender roles",
    "tradblr",
    "tradcat",
    "tradfem",
    "traditional",
    "tradwife",
    "traditional wife",
]
print("Selected Tags: ", tradtags)
# other tags that are weird - "13th", "26th", "31st", "1st"

87
['June', 'modest', 'femininity', '14th', 'cuteanimals', 'animallover', 'mushroomcore', 'forestcore', 'modestclothing', 'meadow', 'farmcore', 'fairycore', 'cute', 'cottagecore aesthetic', '2021', 'cottage core', 'fairy', 'homestead', 'cottagecore', 'farm life', 'grandmacore', 'slow living', 'family', 'April', 'cottage aesthetic', 'homemaker', 'garden', 'grandma', 'vegan', 'honeycore', 'traditional femininity', 'homesteading', 'farming', 'soft', 'oldfashioned', 'cozycore', 'fairy cottage', 'farm', 'feminine style', 'feminine aesthetic', 'country life', 'feminine', 'nature', 'flowers', 'cottage life', 'traditional gender roles', 'tradblr', 'warm and cozy', 'tradcat', '13th', 'May 20th 2021', 'country living', 'April 13th 2021', '26th', 'home', 'naturecore', 'cottagecharm', 'homemaking', 'modesty', 'May', '20th', 'tradfem', 'softcore', 'outfits', 'modestcottagecore', 'housewife', 'prariecore', 'food', 'April 14th 2021', 'traditional', 'forest cottage', 'fairy core', 'cottage', 'cozy cot

In [35]:
# getting all relevant tradwife posts
contains = [not set(tags).isdisjoint(tradtags) for tags in df["tags"]]
traddf = df[contains]
print(len(traddf))

2542
